## Global salinity budget

In [1]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import cartopy.crs as ccrs
import xarray as xr
import cmocean.cm as cmocean
import glob
import matplotlib.path as mpath
import matplotlib.colors as col

import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

from dask.distributed import Client


In [2]:
client = Client()
client


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/40221/status,
Dashboard: /proxy/40221/status,Workers: 7
Total threads: 28,Total memory: 250.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40437,Workers: 7
Dashboard: /proxy/40221/status,Total threads: 28
Started: Just now,Total memory: 250.00 GiB
Comm: tcp://127.0.0.1:43677,Total threads: 4
Dashboard: /proxy/34267/status,Memory: 35.71 GiB
Nanny: tcp://127.0.0.1:37595,


In [3]:
master_session = cc.database.create_session()

control = '01deg_jra55v13_ryf9091'

#session_name = '/g/data/e14/pc5520/model_data/access-om2/basal_melt_MOM5/basal_melt_MOM5.db'
session_name ='/g/data/x77/cy8964/databases/basal_melt_MOM5_v2.db'
#session_name = '/g/data/v45/akm157/model_data/access-om2/basal_melt_MOM5/basal_melt_MOM5.db'
basal_melt_session = cc.database.create_session(session_name)

# temperature of meltwater using Gade line:
basal = 'accessom2-GPC015'
iceberg = 'accessom2-GPC018'


In [4]:
expt = control
session = master_session

geolon_t = cc.querying.getvar(expt,'geolon_t',session, n=1)
geolat_t = cc.querying.getvar(expt,'geolat_t',session, n=1)


In [5]:
# salinity change from restarts

# copy from heat budget notebook (Adele)

# heat content change:
# We need to calculate this from snapshots from the restarts.

salt_dummy = cc.querying.getvar(control, 'salt', master_session, n=1,frequency='1 monthly')

# note I had to collate these restarts first:
filepath = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart995/ocean/ocean_temp_salt.res.nc'
restart1t = xr.open_dataset(filepath)
# only 1 time snapshot:
salt_init = restart1t.salt[0,...]
# fix coords:
salt_init.coords['zaxis_1'] = salt_dummy['st_ocean'].values
salt_init.coords['yaxis_1'] = salt_dummy['yt_ocean'].values
salt_init.coords['xaxis_1'] = salt_dummy['xt_ocean'].values
salt_init = salt_init.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
salt_init = salt_init.drop_vars('Time')

filepath = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart995/ocean/ocean_thickness.res.nc'
restart1z = xr.open_dataset(filepath)
# only 1 time snapshot:
dzt_init = restart1z.rho_dzt[0,...]
# fix coords:
dzt_init.coords['zaxis_1'] = salt_dummy['st_ocean'].values
dzt_init.coords['yaxis_1'] = salt_dummy['yt_ocean'].values
dzt_init.coords['xaxis_1'] = salt_dummy['xt_ocean'].values
dzt_init = dzt_init.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
dzt_init = dzt_init.drop_vars('Time')

filepath = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart1035/ocean/ocean_temp_salt.res.nc'
restart2t = xr.open_dataset(filepath)
# only 1 time snapshot:
salt_end = restart2t.salt[0,...]
# fix coords:
salt_end.coords['zaxis_1'] = salt_dummy['st_ocean'].values
salt_end.coords['yaxis_1'] = salt_dummy['yt_ocean'].values
salt_end.coords['xaxis_1'] = salt_dummy['xt_ocean'].values
salt_end = salt_end.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
salt_end = salt_end.drop_vars('Time')

filepath = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart1035/ocean/ocean_thickness.res.nc'
restart2z = xr.open_dataset(filepath)
# only 1 time snapshot:
dzt_end = restart2z.rho_dzt[0,...]
# fix coords:
dzt_end.coords['zaxis_1'] = salt_dummy['st_ocean'].values
dzt_end.coords['yaxis_1'] = salt_dummy['yt_ocean'].values
dzt_end.coords['xaxis_1'] = salt_dummy['xt_ocean'].values
dzt_end = dzt_end.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
dzt_end = dzt_end.drop_vars('Time')


In [6]:
# compute salt content change:

# J / kg / degC
c_p = 3992.1
rho0 = 1035.0
area = cc.querying.getvar(control, 'area_t', master_session, n=1).load()

# there is a factor of rho_0 in dzt here:
salt_content_change = (1/rho0*((salt_end*dzt_end).sum('st_ocean') - (salt_init*dzt_init).sum('st_ocean'))*area).sum()

print('Net salt content change over this period =',salt_content_change.values/1e12,'x 10^12 g/kg m^3')

Net salt content change over this period = -0.6794126818450127 x 10^12 g/kg m^3


In [7]:
##SURFACE FLUX DURING THIS TIME
# set times to same as prev
area = cc.querying.getvar(control, 'area_t', master_session, n=1).load()

start_time = '2150-01-01'
end_time = '2159-12-31'

In [8]:
sfc_salt_flux_restore_control = cc.querying.getvar(control,'sfc_salt_flux_restore', master_session, 
                                                 start_time = start_time, end_time = end_time) 
sfc_salt_flux_restore_control = sfc_salt_flux_restore_control.sel(time = slice(start_time, end_time))
sfc_salt_flux_restore_control = (sfc_salt_flux_restore_control*sfc_salt_flux_restore_control.time.dt.days_in_month).sum('time')*24*3600 # units kg/m^2 #/sfc_salt_flux_restore_control.time.dt.days_in_month.sum('time')

sfc_salt_flux_ice_control = cc.querying.getvar(control,'sfc_salt_flux_ice', master_session, 
                                                 start_time = start_time, end_time = end_time) 
sfc_salt_flux_ice_control = sfc_salt_flux_ice_control.sel(time = slice(start_time, end_time))
sfc_salt_flux_ice_control = (sfc_salt_flux_ice_control*sfc_salt_flux_ice_control.time.dt.days_in_month).sum('time')*24*3600 # units kg/m^2 #/sfc_salt_flux_ice_control.time.dt.days_in_month.sum('time')

sfc_salt_flux_coupler_control = cc.querying.getvar(control,'sfc_salt_flux_coupler', master_session, 
                                                 start_time = start_time, end_time = end_time) 
sfc_salt_flux_coupler_control = sfc_salt_flux_coupler_control.sel(time = slice(start_time, end_time))
sfc_salt_flux_coupler_control = (sfc_salt_flux_coupler_control*sfc_salt_flux_coupler_control.time.dt.days_in_month).sum('time')*24*3600 # units kg/m^2 #/sfc_salt_flux_ice_control.time.dt.days_in_month.sum('time')


In [9]:
(area*(sfc_salt_flux_ice_control+sfc_salt_flux_restore_control+sfc_salt_flux_coupler_control)).sum().values

array(-1.40604186e+12)

In [10]:
residual = salt_content_change - (area*(sfc_salt_flux_ice_control+sfc_salt_flux_restore_control+sfc_salt_flux_coupler_control)).sum()
print(residual.values)

726629176963.2373


Salt budget = ...

Spill from other nb

In [5]:
def shelf_mask_isobath(var):
    '''
    Masks ACCESS-OM2-01 variables by the region polewards of the 1000m isobath as computed using 
    a script contributed by Adele Morrison.
    Only to be used with ACCESS-OM2-0.1 output!
    '''
    contour_file = np.load('/g/data/ik11/grids/Antarctic_slope_contour_1000m.npz')
    
    shelf_mask = contour_file['contour_masked_above']
    yt_ocean = contour_file['yt_ocean']
    xt_ocean = contour_file['xt_ocean']
    
    # in this file the points along the isobath are given a positive value, the points outside (northwards) 
    # of the isobath are given a value of -100 and all the points on the continental shelf have a value of 0 
    # so we mask for the 0 values 
    shelf_mask[np.where(shelf_mask!=0)] = np.nan
    shelf_mask = shelf_mask+1
    shelf_map = np.nan_to_num(shelf_mask)
    shelf_mask = xr.DataArray(shelf_mask, coords = [('yt_ocean', yt_ocean), ('xt_ocean', xt_ocean)])
    shelf_map = xr.DataArray(shelf_map, coords = [('yt_ocean', yt_ocean), ('xt_ocean', xt_ocean)])
    
    # then we want to multiply the variable with the mask so we need to account for the shape of the mask. 
    # The mask uses a northern cutoff of 59S.
    masked_var = var.sel(yt_ocean = slice(-90, -59.03)) * shelf_mask
    return masked_var, shelf_map


In [8]:
def get_salt_snap(expt,session,start_time):
    salt_dummy = cc.querying.getvar(control, 'salt', master_session, n=1,frequency='1 monthly')
    if start_time == 2150:
        # control snapshot
        filepathsalt = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart995/ocean/ocean_temp_salt.res.nc'
        filepathdzt = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart995/ocean/ocean_thickness.res.nc'
    elif expt == basal:
        outputno = (start_time-2150)*6+996
        filepathsalt = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/accessom2-GPC015/restarts/restart'+str(outputno)+'/ocean/ocean_temp_salt.res.nc'
        filepathdzt = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/accessom2-GPC015/restarts/restart'+str(outputno)+'/ocean/ocean_thickness.res.nc'
    elif expt == iceberg:
        outputno = (start_time-2150)*12+995
        filepathsalt = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/accessom2-GPC018/restarts/restart'+str(outputno)+'/ocean/ocean_temp_salt.res.nc'
        filepathdzt = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/accessom2-GPC018/restarts/restart'+str(outputno)+'/ocean/ocean_thickness.res.nc'
    else:
        print('experiment not found (if control, I have not collated all restarts yet)')
    restart1t = xr.open_dataset(filepathsalt)
    # only 1 time snapshot:
    salt_init = restart1t.salt[0,...]
    # fix coords:
    salt_init.coords['zaxis_1'] = salt_dummy['st_ocean'].values
    salt_init.coords['yaxis_1'] = salt_dummy['yt_ocean'].values
    salt_init.coords['xaxis_1'] = salt_dummy['xt_ocean'].values
    salt_init = salt_init.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
    print(salt_init.Time.values)
    salt_init = salt_init.drop_vars('Time')

    restart1t = xr.open_dataset(filepathsalt)
    # only 1 time snapshot:
    temp_init = restart1t.temp[0,...]
    # fix coords:
    temp_init.coords['zaxis_1'] = salt_dummy['st_ocean'].values
    temp_init.coords['yaxis_1'] = salt_dummy['yt_ocean'].values
    temp_init.coords['xaxis_1'] = salt_dummy['xt_ocean'].values
    temp_init = temp_init.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
    print(temp_init.Time.values)
    temp_init = temp_init.drop_vars('Time')
    
    restart1z = xr.open_dataset(filepathdzt)
    # only 1 time snapshot:
    dzt_init = restart1z.rho_dzt[0,...]
    # fix coords:
    dzt_init.coords['zaxis_1'] = salt_dummy['st_ocean'].values
    dzt_init.coords['yaxis_1'] = salt_dummy['yt_ocean'].values
    dzt_init.coords['xaxis_1'] = salt_dummy['xt_ocean'].values
    dzt_init = dzt_init.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
    print(dzt_init.Time.values)
    dzt_init = dzt_init.drop_vars('Time')

    return salt_init, temp_init, dzt_init

In [10]:
# compute salt content change:

# J / kg / degC
c_p = 3992.1
rho0 = 1035.0
area = cc.querying.getvar(control, 'area_t', master_session, n=1).load()

salt_init, temp_init,dzt_init = get_salt_snap(iceberg,basal_melt_session,2154)
salt_end, temp_end,dzt_end = get_salt_snap(iceberg,basal_melt_session,2155)

# #haline contraction
# depth = -salt_init.st_ocean.values # st_ocean value of the uppermost cell
# geolon_t = cc.querying.getvar(expt,'geolon_t',session, n=1)
# geolat_t = cc.querying.getvar(expt,'geolat_t',session, n=1)
# ## extract coordinate arrays
# yt_ocean = SSS_PSU.yt_ocean.values
# xt_ocean = SSS_PSU.xt_ocean.values
# st_ocean = cc.querying.getvar(expt,'st_ocean',session,n=1).load()
# lon_t = geolon_t
# lat_t = geolat_t
# depth_tile = (lat_t*0+1)*depth
# pressure = xr.DataArray(p_from_z(depth_tile,lat_t), coords = [st_ocean,yt_ocean, xt_ocean], dims = ['st_ocean','yt_ocean', 'xt_ocean'], name = 'pressure', attrs = {'units':'dbar'})
# # convert units to absolute salinity 
# SSS = xr.DataArray(SA_from_SP(salt_init,pressure,lon_t,lat_t), coords = [st_ocean, yt_ocean, xt_ocean], dims = ['time','yt_ocean', 'xt_ocean'], name = 'sea surface salinity', attrs = {'units':'Absolute Salinity (g/kg)'})
# # Compute salt transformation (no density binning)
# haline_contraction_init = xr.DataArray(beta(salt_init, temp_init-273.15, pressure), coords = [st_ocean, yt_ocean, xt_ocean], dims = ['st_ocean','yt_ocean', 'xt_ocean'], name = 'saline contraction coefficient (constant conservative temp)', attrs = {'units':'kg/g'})
# haline_contraction_end = xr.DataArray(beta(salt_end, temp_end-273.15, pressure), coords = [st_ocean, yt_ocean, xt_ocean], dims = ['st_ocean','yt_ocean', 'xt_ocean'], name = 'saline contraction coefficient (constant conservative temp)', attrs = {'units':'kg/g'})


# there is a factor of rho_0 in dzt here:
salt_content_change = (1/rho0*((salt_end*dzt_end).sum('st_ocean') - (salt_init*dzt_init).sum('st_ocean'))*area).sum()
print('Net salt content change over this period =',salt_content_change.values/1e12,'x 10^12 g/kg m^3')

#shelf value
salt_shelf, shelf_mask = shelf_mask_isobath((1/rho0*((salt_end*dzt_end).sum('st_ocean') - (salt_init*dzt_init).sum('st_ocean'))*area))
salt_shelf = salt_shelf.sum()
print('Net salt content change over shelf in this period =',salt_shelf.values/1e12,'x 10^12 g/kg m^3')


# # there is a factor of rho_0 in dzt here:
# FW_content_change = (1/rho0*((haline_contraction_end*salt_end*dzt_end).sum('st_ocean') - (haline_contraction_init*salt_init*dzt_init).sum('st_ocean'))*area).sum()
# print('Net FW content change over this period =',salt_content_change.values/1e12,'x 10^12 kg m^3')

# #shelf value
# FW_shelf, shelf_mask = shelf_mask_isobath((1/rho0*((haline_contraction_end*salt_end*dzt_end).sum('st_ocean') - (haline_contraction_init*salt_init*dzt_init).sum('st_ocean'))*area))
# FW_shelf = FW_shelf.sum()
# print('Net FW content change over shelf in this period =',FW_shelf.values/1e12,'x 10^12 kg m^3')


1.0
1.0
1.0
1.0
1.0
1.0
Net salt content change over this period = -91.27927486910308 x 10^12 g/kg m^3
Net salt content change over shelf in this period = -11.334103352813402 x 10^12 g/kg m^3


In [11]:
# compute salt content change:

# J / kg / degC
c_p = 3992.1
rho0 = 1035.0
area = cc.querying.getvar(control, 'area_t', master_session, n=1).load()

salt_init, temp_init,dzt_init = get_salt_snap(basal,basal_melt_session,2154)
salt_end, temp_end,dzt_end = get_salt_snap(basal,basal_melt_session,2155)

# #haline contraction
# depth = -salt_init.st_ocean.values # st_ocean value of the uppermost cell
# geolon_t = cc.querying.getvar(expt,'geolon_t',session, n=1)
# geolat_t = cc.querying.getvar(expt,'geolat_t',session, n=1)
# ## extract coordinate arrays
# yt_ocean = SSS_PSU.yt_ocean.values
# xt_ocean = SSS_PSU.xt_ocean.values
# st_ocean = cc.querying.getvar(expt,'st_ocean',session,n=1).load()
# lon_t = geolon_t
# lat_t = geolat_t
# depth_tile = (lat_t*0+1)*depth
# pressure = xr.DataArray(p_from_z(depth_tile,lat_t), coords = [st_ocean,yt_ocean, xt_ocean], dims = ['st_ocean','yt_ocean', 'xt_ocean'], name = 'pressure', attrs = {'units':'dbar'})
# # convert units to absolute salinity 
# SSS = xr.DataArray(SA_from_SP(salt_init,pressure,lon_t,lat_t), coords = [st_ocean, yt_ocean, xt_ocean], dims = ['time','yt_ocean', 'xt_ocean'], name = 'sea surface salinity', attrs = {'units':'Absolute Salinity (g/kg)'})
# # Compute salt transformation (no density binning)
# haline_contraction_init = xr.DataArray(beta(salt_init, temp_init-273.15, pressure), coords = [st_ocean, yt_ocean, xt_ocean], dims = ['st_ocean','yt_ocean', 'xt_ocean'], name = 'saline contraction coefficient (constant conservative temp)', attrs = {'units':'kg/g'})
# haline_contraction_end = xr.DataArray(beta(salt_end, temp_end-273.15, pressure), coords = [st_ocean, yt_ocean, xt_ocean], dims = ['st_ocean','yt_ocean', 'xt_ocean'], name = 'saline contraction coefficient (constant conservative temp)', attrs = {'units':'kg/g'})


# there is a factor of rho_0 in dzt here:
salt_content_change = (1/rho0*((salt_end*dzt_end).sum('st_ocean') - (salt_init*dzt_init).sum('st_ocean'))*area).sum()
print('Net salt content change over this period =',salt_content_change.values/1e12,'x 10^12 g/kg m^3')

#shelf value
salt_shelf, shelf_mask = shelf_mask_isobath((1/rho0*((salt_end*dzt_end).sum('st_ocean') - (salt_init*dzt_init).sum('st_ocean'))*area))
salt_shelf = salt_shelf.sum()
print('Net salt content change over shelf in this period =',salt_shelf.values/1e12,'x 10^12 g/kg m^3')


# # there is a factor of rho_0 in dzt here:
# FW_content_change = (1/rho0*((haline_contraction_end*salt_end*dzt_end).sum('st_ocean') - (haline_contraction_init*salt_init*dzt_init).sum('st_ocean'))*area).sum()
# print('Net FW content change over this period =',salt_content_change.values/1e12,'x 10^12 kg m^3')

# #shelf value
# FW_shelf, shelf_mask = shelf_mask_isobath((1/rho0*((haline_contraction_end*salt_end*dzt_end).sum('st_ocean') - (haline_contraction_init*salt_init*dzt_init).sum('st_ocean'))*area))
# FW_shelf = FW_shelf.sum()
# print('Net FW content change over shelf in this period =',FW_shelf.values/1e12,'x 10^12 kg m^3')


1.0
1.0
1.0
1.0
1.0
1.0
Net salt content change over this period = -47.7838434579509 x 10^12 g/kg m^3
Net salt content change over shelf in this period = -7.180159619344305 x 10^12 g/kg m^3
